In [12]:

import cv2
from scipy import ndimage, misc
from scipy.signal import argrelextrema
from matplotlib import pyplot as plt
import rawpy
from PIL import Image
import cv2 as cv
import numpy as np
import math
from skimage.transform import pyramid_gaussian
from scipy.signal import find_peaks
import time
import imutils
import sys


def makeclippingmask(image):
    

    #makes a clipping mask around each bright spot so the analysis isn't thrown off
    grayA = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    blurred = cv.GaussianBlur(grayA, (11,11), 0)

    #watershed thresholding. Based on: https://docs.opencv.org/3.4/d2/dbd/tutorial_distance_transform.html
    src = image.copy()
    
    
    # Create a kernel that we will use to sharpen our image
    # an approximation of second derivative, a quite strong kernel
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)
    # do the laplacian filtering as it is
    # well, we need to convert everything in something more deeper then CV_8U
    # because the kernel has some negative values,
    # and we can expect in general to have a Laplacian image with negative values
    # BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
    # so the possible negative number will be truncated
    imgLaplacian = cv.filter2D(src, cv.CV_32F, kernel)
    sharp = np.float32(src)
    imgResult = sharp - imgLaplacian
    # convert back to 8bits gray scale
    imgResult = np.clip(imgResult, 0, 255)
    imgResult = imgResult.astype('uint8')
    imgLaplacian = np.clip(imgLaplacian, 0, 255)
    imgLaplacian = np.uint8(imgLaplacian)
    #cv.imshow('Laplace Filtered Image', imgLaplacian)
    #cv.imshow('New Sharped Image', imgResult)
    
    # Create binary image from source image
    # Create binary image from source image
    bw = cv.cvtColor(imgResult, cv.COLOR_BGR2GRAY)
   # _, bw2 = cv.threshold(grayA, 30, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    #plt.imshow(bw)
    bw = cv.adaptiveThreshold(bw, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 37, 1)
    #cv.imshow('Binary Image', bw)
    
    
    opening = cv.morphologyEx(bw,cv.MORPH_OPEN,kernel, iterations = 3)
    
    
    
    # sure background area
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    
    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    # Threshold to obtain the peaks
    # This will be the markers for the foreground objects
    _, sure_fg = cv.threshold(dist_transform, 0.2, 1.0, cv.THRESH_BINARY)
    # Dilate a bit the dist image
    kernel1 = np.ones((3,3), dtype=np.uint8)
    sure_fg = cv.dilate(sure_fg, kernel1)
    ret, markers = cv.connectedComponents(np.uint8(sure_fg))
    
    
    #cv.imshow('Final Result', sure_fg)
    # Finding unknown region
    sure_fg = np.float32(sure_fg)
    sure_bg = np.float32(sure_bg)
    unknown = cv.subtract(sure_bg,sure_fg)
    
    #cv.imshow('Distance Transform Image', dist_transform)
    
    # Marker labelling
    

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    #print(markers.shape)
    
    markers = cv.watershed(imgResult,markers)
    #print(markers)
    
    mask2 = np.zeros(image.shape[:2], dtype= np.uint8)
    mask2[markers >1] = [255]
    
    #colours=((255,255,255))
    # Fill labeled objects with random colors
    #for i in range(markers.shape[0]):
    #    for j in range(markers.shape[1]):
    #        index = markers[i,j]
    #        if index>0:
    #            mask2[i,j] = 255
    

    image_rgb=image.copy()
    image_blocked = cv.bitwise_and(image_rgb, image_rgb, mask=mask2)
    #plt.imshow(image_blocked)
    return image_blocked

def imgregfun(imagebef, imageafter):
#### A function for image registration, stolen of the internet but I can't remember where from
###Inputs: imagebef- the before image
##########imageafter- the after image
###outputs: transimaf- the translated after image
    # Open the image files.
    img1_color = imageafter  # Image to be aligned.
    img2_color = imagebef  # Reference image.

    # Convert to grayscale.
    img1 = cv.cvtColor(img1_color, cv.COLOR_BGR2GRAY)
    img2 = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)
    height, width = img2.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not reqiured in this case).
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)

    # Match features between the two images.
    # We create a Brute Force matcher with
    # Hamming distance as measurement mode.
    matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Match the two sets of descriptors.
    matches = matcher.match(d1, d2)

    # Sort matches on the basis of their Hamming distance.
    matches.sort(key=lambda x: x.distance)

    # Take the top 90 % matches forward.
    matches = matches[:np.int(len(matches) * 90)]
    no_of_matches = len(matches)

    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
        p1[i, :] = kp1[matches[i].queryIdx].pt
        p2[i, :] = kp2[matches[i].trainIdx].pt

    # Find the homography matrix.
    homography, mask = cv.findHomography(p1, p2, cv.RANSAC)

    # Use this matrix to transform the
    # colored image wrt the reference image.
    transformed_img = cv.warpPerspective(img1_color, homography,
                                          (width, height))
    transimaf=transformed_img
    return transimaf
def imageprocessingfunction(beforefol,afterfol):
###A function for getting all the CR2 files within the before and after folders, then reading them
### and saving them on the disk as virtual images 
##########################################################################
###Inputs: beforefol: selected before folder
##########afterfol: selected after folder 
###Outputs: imaf: the images in the after folder as an array
###########imbef: the images in the before folder as an array 
###########beforeimfile: the list of before image files
###########afterimfile: the list of after image files
    # Get file list
    beforeimfile=glob.glob(beforefol+"\\"+"*.CR2")
    afterimfile=glob.glob(afterfol+"\\"+"*.CR2")
    #print(afterimfile)

    #Exifdata is just there in case you need to edit the images in a fancy way.
    imaf,labaf,imbef,labef=[],[],[],[]
    for impath in afterimfile:
        image,exifdata=   convertfilefun(impath)
        imaf.append(np.dstack((image)))
        labaf.append(exifdata)
    for impath in beforeimfile:
        image,exifdata= convertfilefun(impath)
        imbef.append(np.dstack((image)))
        labef.append(exifdata)
    return imaf,imbef,beforeimfile,afterimfile

def convertfilefun(path):
## a function which converts CR2 images to TIFF images the computer can actually read
## input: path- path to raw image
## output : an image that is readable using cv2
    with rawpy.imread(path) as raw:
        #Can fiddle with camera settings but I wouldn't reccoment it
        rgb = raw.postprocess(use_camera_wb=True,
                              no_auto_bright=True,
                              gamma=(2.222, 4.5),
                              chromatic_aberration=(1, 1))
        #cv2.imwrite(path + '.tiff',rgb)
        # extract EXIF data to save as metadata
        metdat = Image.open(path)
        exifdata = metdat.getexif()
        image = rgb
        image = rgb.reshape(
            (1, image.shape[0], image.shape[1], image.shape[2]))
        return image, exifdata
        #plt.imsave(path + '.png',rgb)
        #g=print(path + '.png')
        #return g


def gamma_correction(image_rgb):
    hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    hue, sat, val = cv2.split(hsv)
    

    # compute gamma = log(mid*255)/log(mean)
    mid = 0.5
    mean = np.mean(val)
    gamma = math.log(mid*255)/math.log(mean)
    

    # do gamma correction on value channel
    val_gamma = np.power(val, gamma).clip(0,255).astype(np.uint8)
    hsv_gamma = cv2.merge([hue, sat, val_gamma])
    img_gamma2 = cv2.cvtColor(hsv_gamma, cv2.COLOR_HSV2RGB)
    return img_gamma2

def grayscale_formulation(image_gamma2):
    k_b=0.114
    k_f=0.299
    r,g,b=cv2.split(image_gamma2)
    Y=k_f*r+(1-k_f-k_b)*g+k_b*b
    Cb=-0.168736*r-0.331264*g+0.5*b
    Cr=0.5*r-0.418688*g+0.081312*b
    img = cv.merge((Y, Cb, Cr))
    return img




def gaussian(x,y,z):
    g=((1/(2*3.1415*z**2))*exp(-(x**2+y**2)/(2*z**2)))
    return g
def searchforextrema(results1):
    extrema_max=np.array([argrelextrema(results1, np.greater) for window in results1])
    extrema_min=np.array([argrelextrema(results1, np.less) for window in results1])
    return extrema_max,extrema_min
def findglobalmaxima(extrema_max1,extrema_min1,extrema_max2,extrema_min2):
    results=1


def laplacianfiltering(image_gray):
    Y,C_b,C_r=cv.split(image_gray)
    
    
    result1 = ndimage.gaussian_laplace(Y, sigma=1)
    result2 = ndimage.gaussian_laplace(Y, sigma=8)
    extrama1=result2-result1
    
    #result3 = ndimage.gaussian_laplace(Y, sigma=8)
    result3 = ndimage.gaussian_laplace(Y, sigma=16)
    extrema2=result3-result2
    
    result5 = ndimage.gaussian_laplace(Y, sigma=32)
    
    extrema3=result5-result3
    return extrama1,extrema2,extrema3

def imagepyramid(image, scale):
    rows, cols, dim = image.shape
    pyramid = tuple(pyramid_gaussian(image, downscale=scale))
    for p in pyramid[1:]:
        # if the image is too small, break from the loop
        if p.shape[0] < 30 or p.shape[1] < 30:
            break
        # show the resized image
        plt.imshow("Layer", p)
        cv2.waitKey(0)
def sliding_window(image, stepSize, windowSize):
    for y in list(range(0, image.shape[1], stepSize)):
        for x in list(range(0, image.shape[1], stepSize)):
            #print("we're in x")
            cropim=image[y:y + windowSize[1], x:x + windowSize[0]]
            yield cropim
            

            
            
def newsliding_window(image,stepSize,windowSize):
    image_windows=[]
    for y in list(range(0, image.shape[0], stepSize)):
        for x in list(range(0, image.shape[1], stepSize)):
            image_crop=image[y:y + windowSize[1], x:x + windowSize[0]]
            image_windows.append(image_crop)
    #print(len(image_windows))
            
    return image_windows
    
    
def pyramid(image, scale, minSize):
    yield image
    
    for  resized in pyramid_gaussian(image, downscale=scale):
        # if the image is too small, break from the loop
        #print(resized.shape[1])
        if resized.shape[0] < minSize or resized.shape[1] < minSize:
            break
        else:
            yield resized
    

            
def count_dist_peaks(series, bins, prominence, width):
    count, division = np.histogram(series, bins=bins)
    peaks, props = find_peaks(count, prominence=prominence, width=width)
    return peaks

def some_func(window):
    channel=cv.split(window)
    #print(np.min(channel[0]))
    for i,colour in enumerate(channel):
        hist = cv2.calcHist([colour], [0], None, [8], [0, 256])


        if imutils.is_cv2():
            hist = cv2.normalize(hist).flatten()
    # otherwise handle for OpenCV 3+
        else:
            hist = cv2.normalize(hist, hist).flatten()
        if i==0:
            Y_features=hist
            #print(Y_features)

        elif i==1:
            Cr_features=hist

        else:
            Cb_features=hist

    return [Y_features,Cr_features,Cb_features]

def some_func2(window):
    
    hist = cv2.calcHist([window], [0], None, [8], [0, 256])


    if imutils.is_cv2():
        hist = cv2.normalize(hist).flatten()
# otherwise handle for OpenCV 3+
    else:
        hist = cv2.normalize(hist, hist).flatten()
    

    return [hist]




def sliding_window_histogram(resized):
    
    # loop over the sliding window for each layer of the pyramid
    Y_features=[]
    Cr_features=[]
    Cb_features=[]
   # windows=sliding_window(resized, 4, (100, 100))
    img_r,img_g,img_b=cv.split(resized.astype('uint8'))
    hmax,hmin=scipyslidinghistogram(img_r)
    return hmax,hmin
    


    
    
    
    
from skimage.filters.rank import windowed_histogram
from skimage.morphology import disk, ball

def scipyslidinghistogram(img):
    img.astype('uint8')
    hist_img = windowed_histogram(img, disk(18),n_bins=8)
    hmax,hmin=searchforextrema(hist_img)
    return hmax,hmin
    
    


        
    #for window in windows:




            
    
        #print(Y_features)
        #Y_max,Y_min=searchforextrema(np.array(Y_features))
        
     
        #Cr_max,Cr_min=searchforextrema(np.array(Cr_features))
        #Cb_max,Cb_min=searchforextrema(np.array(Cb_features))
    return output_img#Y_max,Y_min,Cr_max,Cr_min,Cb_max,Cb_min

    
    


In [13]:
image_before=r"D:\OneDrive - UNSW\Image_Analysis\NewMicroscope\OneDrive_2021-06-28\Time series experiment for analysis\Test\slide1\initial\satellite\IMG_0380s1.CR2"
image_after=r"D:\OneDrive - UNSW\Image_Analysis\NewMicroscope\OneDrive_2021-06-28\Time series experiment for analysis\Test\slide1\initial\target\IMG_0580t20.CR2"

image_bef,exifdata= convertfilefun(image_before)
image_af,exifdata= convertfilefun(image_after)
%matplotlib qt




image_bef= makeclippingmask(image_bef[0])
image_af= makeclippingmask(image_af[0])

transimaf=imgregfun(image_bef, image_af)

gammaimg_bef=gamma_correction(image_bef)
gammaimg_af=gamma_correction(transimaf)


image_bef_gray=grayscale_formulation(gammaimg_bef)
image_af_gray=grayscale_formulation(gammaimg_af)

#extrema1,extrema2,extrema3=laplacianfiltering(image_bef_gray)
y_max_tot=[]
y_min_tot=[]
cr_max_tot=[]
cr_min_tot=[]
cb_max_tot=[]
cb_min_tot=[]

images=pyramid(gammaimg_bef, 3,40)

for resized in images:
    #plt.imshow(resized)
    #Y_max,Y_min,Cr_max,Cr_min,Cb_max,Cb_min= sliding_window_histogram(resized)
    #print(Y_max)
    hmax,hmin=sliding_window_histogram(resized)
    pausehere=here

    print("Got here")

            
            
            
        










c:\program files\python36\lib\site-packages\ipykernel_launcher.py:230: RuntimeWarning: divide by zero encountered in power


NameError: name 'here' is not defined

In [ ]:


def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    
    rangey=[0,1,2,3]
    for y in list(range(0, image.shape[1], stepSize)):
        
        print(y)
        for x in list(range(0, image.shape[1], stepSize)):
            print(x)
            # yield the current window
            #plt.imshow(image[y:y + windowSize[1], x:x + windowSize[0]])
            return (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

y=newsliding_window(resized,8,(100,100))
plt.imshow(windows[0])

In [ ]:
print(output_img)

In [ ]:
 sliding_window_histogram(resized)